![Evidence 3](https://i.imgur.com/mu6ZuGT.jpg)
# **Maestría en Inteligencia Artificial Aplicada**

## Curso: **Procesamiento de Lenguaje Natural**

### Tecnológico de Monterrey

### Prof Luis Eduardo Falcón Morales

## Actividad Semana 5

### **Vectores Embebidos Pre-entrenados: Fasttext**

#### **Nombres y matrículas de los integrantes del equipo:**



*   **Julio Baltazar Colín: A01794476**
*   Elemento de lista



In [ ]:
# Aquí deberás incluir todas las librerías que requieras durante esta actividad:
import pandas as pd
import numpy as np

import nltk
from nltk.corpus import stopwords

import re
import string

from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('punkt')

# Instalar el paquete contractions
!pip install contractions
from collections import Counter
import nltk
import contractions
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import pickle
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


##**Pregunta - 1:**



Descarga los 3 archivos de Canvas y genera un nuevo DataFrame de Pandas con ellos.

**Llama simplemente "df" a dicho DataFrame.**

Los archivos los encuentras en Canvas: amazon5.txt, imdb5.txt, yelp5.txt.



In [ ]:
#carga de datos desde un repositorio externo:

import requests
urlAmzn = 'https://raw.githubusercontent.com/Julio21-ai/NLP-GPO10/main/5.2%20Vectores%20Embebidos%20Pre-entrenados/amazon5.txt'
urlimdb = 'https://raw.githubusercontent.com/Julio21-ai/NLP-GPO10/main/5.2%20Vectores%20Embebidos%20Pre-entrenados/imdb5.txt'
urlyelp = 'https://raw.githubusercontent.com/Julio21-ai/NLP-GPO10/main/3.2%20Actividad%202/yelp_labelled.txt'

responce_amzn = requests.get(urlAmzn)
responce_imdb = requests.get(urlimdb)
responce_urlyelp = requests.get(urlyelp)

data_amzn = None
data_imdb= None
data_urlyelp= None

if responce_amzn.status_code == 200: #200 = OK
    data_amzn = responce_amzn.text
else:
    print("Error al obtener el archivo:", responce_amzn.status_code)

if responce_imdb.status_code == 200: #200 = OK
    data_imdb = responce_imdb.text
else:
    print("Error al obtener el archivo:", responce_imdb.status_code)

if responce_urlyelp.status_code == 200: #200 = OK
    data_urlyelp = responce_urlyelp.text
else:
    print("Error al obtener el archivo:", responce_imdb.status_code)


In [ ]:

# ******* Inlcuye a continuación todas las líneas de código y celdas que requieras: ***********
from io import StringIO

dfa = pd.read_csv(StringIO(data_amzn), sep='\t', names=['review','label'], header=None, encoding='utf-8')
dfi = pd.read_csv(StringIO(data_imdb), sep=r'\s{3,}', names=['review','label'], header=None, encoding='utf-8')
dfy = pd.read_csv(StringIO(data_urlyelp), sep='\t', names=['review','label'], header=None, encoding='utf-8')

print('Total de registros de Amazon:',dfa.shape)
print('Total de registros de IMBD:',dfi.shape)
print('Total de registros de Yelp:',dfy.shape)

df = pd.concat([dfa, dfi, dfy], ignore_index=True)
df['label'] = pd.to_numeric(df['label'])

print('Total de registros dataframe df:',df.shape)

# *********** Aquí termina la sección de agregar código *************


Total de registros de Amazon: (1000, 2)
Total de registros de IMBD: (1000, 2)
Total de registros de Yelp: (1000, 2)
Total de registros dataframe df: (3000, 2)


<ipython-input-4-c6ff799b6a37>:5: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  dfi = pd.read_csv(StringIO(data_imdb), sep=r'\s{3,}', names=['review','label'], header=None, encoding='utf-8')


In [ ]:
# Verifiquemos la información del DataFrame:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   review  3000 non-null   object
 1   label   3000 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 47.0+ KB


In [ ]:
# Y veamos sus primeros registros:
df.head()

,review,label
0,So there is no way for me to plug it in here i...,0
1,"Good case, Excellent value.",1
2,Great for the jawbone.,1
3,Tied to charger for conversations lasting more...,0
4,The mic is great.,1


##**Pregunta - 2:**

Realiza el proceso de limpieza.

Aplica el preprocesamiento que consideres adecuado, sin embargo, deberás aplicar necesariamente alguna de las técnicas de lematización.











In [ ]:
# ******* Inlcuye a continuación todas las líneas de código y celdas que requieras: ***********
def remover_espacios_extra(texto):
    # Eliminar los espacios iniciales y finales y dividir el texto por espacios
    texto_filtrado = re.sub(r'\s+', ' ', texto).strip().split()
    # texto_filtrado las palabras con un solo espacio
    retval = ' '.join(texto_filtrado)
    return retval

def procesar_comilla(texto):
    texto_filtrado = texto
    # Eliminar terminaciones 's
    texto_filtrado = re.sub(r"'s\b", '', texto_filtrado)
    # Reemplazar ".", "," , ":", y "_" por espacios en blanco
    texto_filtrado = re.sub(r"\'", ' ', texto_filtrado)
    return texto_filtrado

def eliminar_no_alfa(texto):
    texto_filtrado = texto

    # Eliminar caracteres no alfanuméricos, excepto ".", ",", ":", "-" y "_"
    texto_filtrado = re.sub(r"[^\w\s\.,:-]+", '', texto_filtrado)

    # Eliminar guiones repetidos en medio de la cadena
    texto_filtrado = re.sub(r'(?<!-)-{2,}(?!-)', ' ', texto_filtrado)

    # Eliminar guiones al principio y al final de la cadena
    texto_filtrado = re.sub(r'^-+|-+$', '', texto_filtrado)

    # Reemplazar ".", "," , ":", y "_" por espacios en blanco
    texto_filtrado = re.sub(r'[\.,:_]', ' ', texto_filtrado)

    return texto_filtrado

def minusculas(texto):
    #Convertir a minusculas
    return texto.lower()

def remover_caracteres_sueltos(texto):
    # Expresión regular para encontrar caracteres sueltos
    expresion_regular = r'\b\w{1}\b'  # Coincidirá con cualquier carácter único

    # Reemplaza los caracteres sueltos con una cadena vacía
    texto_filtrado = re.sub(expresion_regular, '', texto)

    return texto_filtrado

def eliminar_no_ascii(texto):
    # Eliminar caracteres no ASCII
    retval = re.sub(r'[^\x00-\x7F]+', '', texto)
    return  texto

def eliminar_digitos(texto):
    # Utilizar una expresión regular para eliminar palabras con dígitos
    texto_filtrado = re.sub(r'\b\w*\d\w*\b', '', texto)

    # Eliminar espacios adicionales resultantes
    #texto_filtrado = re.sub(r'\s+', ' ', texto_filtrado).strip()

    return texto_filtrado


def expandir_contracciones(texto):
    # Expandir contracciones
    return contractions.fix(texto)


def lematizado(texto):
    wnl = WordNetLemmatizer()
    # Tokenizar el texto en palabras
    palabras = word_tokenize(texto)

    palabras_lemmatizadas = []
    for palabra in palabras:
        # Lemmatizar la palabra como adjetivo
        lemmatizada = wnl.lemmatize(palabra, 'a')
        # Lemmatizar la palabra como verbo
        lemmatizada = wnl.lemmatize(lemmatizada, 'v')
        # Lemmatizar la palabra como sustantivo
        lemmatizada = wnl.lemmatize(lemmatizada, 'n')
        # Lemmatizar la palabra como adverbio
        lemmatizada = wnl.lemmatize(lemmatizada, 'r')
        palabras_lemmatizadas.append(lemmatizada)

    # Unir las palabras lemmatizadas en un texto de nuevo
    texto_filtrado = ' '.join(palabras_lemmatizadas)

    return texto_filtrado

def remove_stop_words(lista_palabras, stopwords):
    #Eliminar las stopwords
    palabras_sin_stopwords = [palabra for palabra in lista_palabras if palabra not in stopwords]
    return palabras_sin_stopwords


def tokenizar(enunciado):
    # Tokenizar el texto
    tokens = word_tokenize(enunciado)
    # Eliminar caracteres no alfanuméricos
    tokens = [token for token in tokens if token.isalnum()]
    return tokens

def limpiar_texto(texto):
  texto_filtrado = texto
  texto_filtrado = expandir_contracciones(texto_filtrado)
  texto_filtrado = procesar_comilla(texto_filtrado)
  texto_filtrado = eliminar_no_alfa(texto_filtrado)
  texto_filtrado = eliminar_no_ascii(texto_filtrado)
  texto_filtrado = eliminar_digitos(texto_filtrado)
  texto_filtrado = remover_espacios_extra(texto_filtrado)
  texto_filtrado = lematizado(texto_filtrado)
  texto_filtrado = remover_caracteres_sueltos(texto_filtrado)
  texto_filtrado = minusculas(texto_filtrado)
  return texto_filtrado

# *********** Aquí termina la sección de agregar código *************



In [ ]:

# ******* Inlcuye a continuación todas las líneas de código y celdas que requieras: ***********

X = df.review     # Serie de strings
Y = df.label      # Serie de enteros 0s y 1s


texto_limpio = X.apply(limpiar_texto)
df['review'].to_csv('rsultado_sucio.txt', index=False)
texto_limpio.to_csv('resultado_limpio.txt', index=False)

tokens_list = [tokenizar(texto) for texto in texto_limpio]
Xclean = tokens_list

# *********** Aquí termina la sección de agregar código *************

In [ ]:
# Despleguemos los primeros comentarios después de tu proceso de limpieza:

for x in Xclean[0:5]:
  print(x)


['so', 'there', 'be', 'no', 'way', 'for', 'me', 'to', 'plug', 'it', 'in', 'here', 'in', 'the', 'us', 'unless', 'go', 'by', 'converter']
['good', 'case', 'excellent', 'value']
['great', 'for', 'the', 'jawbone']
['tied', 'to', 'charger', 'for', 'conversation', 'last', 'more', 'than', 'minute', 'major', 'problems']
['the', 'mic', 'be', 'great']


#**Pregunta - 3:**



Realicemos una partición aleatoria con los mismos porcentajes de la práctica pasada para poder comparar dichos resultados con los de
esta actividad, a saber, 70%, 15% y 15%, para entrenamiento, validación y prueba, respectivamente.

In [ ]:

# ************* Inicia la sección de agregar código:*****************************

from sklearn.model_selection import train_test_split

x_train, x_val_and_test, y_train, y_val_and_test = train_test_split(Xclean, Y, train_size=.70, shuffle=True, random_state=1)
x_val, x_test, y_val, y_test = train_test_split(x_val_and_test, y_val_and_test, test_size=.50, shuffle=True, random_state=17)

print('X,y Train:', len(x_train), len(y_train))      # los "x_" son "list" y los "y_" son "Series"
print('X,y Val:', len(x_val), len(y_val))
print('X,y Test', len(x_test), len(y_test))

# *********** Termina la sección de agregar código *************


# verificemos las dimensiones obtenidas:
print('X,y Train:', len(x_train), len(y_train))
print('X,y Val:', len(x_val), len(y_val))
print('X,y Test', len(x_test), len(y_test))

X,y Train: 2100 2100
X,y Val: 450 450
X,y Test 450 450
X,y Train: 2100 2100
X,y Val: 450 450
X,y Test 450 450


#**Pregunta - 4:**



Construye tu vocabulario a continuación


In [ ]:
# a.	Usa el conjunto de entrenamiento para generar tu vocabulario
#     con un tamaño que consideres adecuado:


# ******* Inlcuye a continuación todas las líneas de código y celdas que requieras: ***********


lista_tokens = Counter()

for k in range(len(x_train)):
  lista_tokens.update(x_train[k])


# Contar la frecuencia de cada token
frecuencia_tokens = Counter(lista_tokens)

# Definir el umbral de frecuencia
umbral_frecuencia = 2

# Filtrar palabras con frecuencia menor al umbral
frecuencia_filtrada = {token: freq for token, freq in frecuencia_tokens.items() if freq >= umbral_frecuencia}

# Ordenar palabras por frecuencia
palabras_ordenadas = sorted(frecuencia_filtrada.items(), key=lambda item: item[1], reverse=True)

# Crear el vocabulario: un diccionario que asigna un índice único a cada token
diccionario = {token: idx for idx, (token, _) in enumerate(palabras_ordenadas)}

# *********** Aquí termina la sección de agregar código *************

In [ ]:
# b.	Indica el tamaño del vocabulario generado.

print('Longitud del vocabulario generado:')


# ******* Inicia la sección de agregar código: ***********
print(f"{len(diccionario)}, palabras")

N = 10  # Número de elementos a imprimir
print(f"Primeros {N} elementos del diccionario:")
for i, (palabra, idx) in enumerate(diccionario.items()):
    if i >= N:
        break
    print(f"Token: {palabra}, Índice: {idx}")
# *********** Aquí termina la sección de agregar código *************

Longitud del vocabulario generado:
1495, palabras
Primeros 10 elementos del diccionario:
Token: be, Índice: 0
Token: the, Índice: 1
Token: and, Índice: 2
Token: it, Índice: 3
Token: to, Índice: 4
Token: this, Índice: 5
Token: not, Índice: 6
Token: of, Índice: 7
Token: have, Índice: 8
Token: in, Índice: 9


c.	¿Por qué debe usarse solamente el conjunto de entrenamiento para generar el vocabulario?


### ++++++++ Inicia la sección de agregar texto: +++++++++++

La principal razón para usar el conjunto de entrenamiento para generar el vocabulario es evitar el filtrado de información, lo que impide que los modelos tengan una ventaja al momento de realizar las predicciones. Esto previene el sesgo en la información y proporciona una mayor imparcialidad en el modelo.

Otra razón es mantener la consistencia al emplear un único diccionario que sea común para todos los modelos, facilitando el manejo de la información y permitiendo que los datos sean reproducibles.

El usar el vocabulario para el conjunto de entrenamiento tambien es mas eficiente pues permite manejar un unico conjunto de datos, en los casos donde se trabaja con modelos grandes, el costo computacional resulta menor.

Por ultimo si cada conjunto manejara su propio vocabulario, seria muy dificil la convergencia de los modelos pues se tendrian varias palabras que estaria fuera del vocabulario conocido para el modelo lo que haria muy dificil su convergencia.


### ++++++++ Termina la sección de agregar texto: +++++++++++


In [ ]:
# d.	Con el vocabulario generado, filtra los conjuntos de entrenamiento,
#     validación y prueba para que todos los comentarios usen solamente las
#     palabras de este vocabulario.

#     Llamar train_x, val_x y test_x a estos tres conjuntos.


# ******* Inlcuye a continuación todas las líneas de código y celdas que requieras: ***********

train_x = []
for ss in x_train:
  train_x.append([w for w in ss if w in diccionario])

val_x = []
for ss in x_val:
  val_x.append([w for w in ss if w in diccionario])

test_x = []
for ss in x_test:
  test_x.append([w for w in ss if w in diccionario])

# *********** Aquí termina la sección de agregar código *************


In [ ]:
# Vemos el resultado de los primeros comentarios del conjunto de entrenamiento:

for ss in train_x[0:5]:
  print(ss)

['his', 'do', 'not', 'fare', 'much', 'good', 'with', 'people', 'like', 'morgan', 'and', 'ed', 'just', 'waste']
['tonight', 'have', 'the', 'filet', 'special', 'and', 'it', 'suck']
['pay', 'the', 'bill', 'but', 'do', 'not', 'tip', 'because', 'felt', 'the', 'server', 'do', 'terrible', 'job']
['how', 'can', 'you', 'call', 'yourself', 'if', 'you', 'can', 'not', 'properly', 'cook', 'steak', 'do', 'not', 'understand']
['however', 'the', 'keypad', 'be', 'so', 'tinny', 'that', 'sometimes', 'the', 'wrong', 'button']


#**Pregunta - 5:**


a. Incluye una tabla comparativa de pros y contras entre los modelos FastText, word2vec de Google y Glove de Stanford.

### ++++++++ Inicia la sección de agregar texto: +++++++++++

<table>
  <thead>
    <tr>
      <th>Modelo</th>
      <th>Pros</th>
      <th>Contras</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>FastText</td>
      <td>
        <ul>
          <li>Captura información subpalabra, lo que ayuda con palabras raras y OOV (Out Of Vocabulary).</li>
          <li>Más preciso en lenguajes morfológicamente ricos.</li>
          <li>Capaz de manejar palabras compuestas y errores tipográficos.</li>
          <li> Captura mejor la relación entre palabras similares por morfología.</li>
          <li>Open source.</li>
          <li>Disponible en varios idiomas.</li>          
        </ul>
      </td>
      <td>
        <ul>
          <li>Generalmente más lento de entrenar comparado con Word2Vec.</li>
          <li>Requiere más espacio de almacenamiento debido a la información de subpalabra.</li>
        </ul>
      </td>
    </tr>
    <tr>
      <td>Word2Vec (Google)</td>
      <td>
        <ul>
          <li>Muy eficiente y rápido de entrenar.</li>
          <li>Genera vectores de alta calidad que capturan relaciones semánticas.</li>
          <li>Disponible en muchas implementaciones y bibliotecas.</li>
        </ul>
      </td>
      <td>
        <ul>
          <li>No maneja palabras raras o OOV sin renentrenar.</li>
          <li>No captura información subpalabra.</li>
          <li>Trata las palabras como unidades atomicas por lo que en ocaciones pude perder el contexto.</li>
          <li>Version opensource reducida, versiones comerciales bajo licencia.</li>
        </ul>
      </td>
    </tr>
    <tr>
      <td>GloVe (Stanford)</td>
      <td>
        <ul>
          <li>Basado en estadísticas globales, lo que puede capturar información de contexto mejor.</li>
          <li>Genera vectores que pueden ser más precisos para ciertas tareas NLP.</li>
          <li>Muy eficiente para preentrenar en grandes corpus.</li>
        </ul>
      </td>
      <td>
        <ul>
          <li>Generalmente más lento de entrenar comparado con Word2Vec.</li>
          <li>El rendimiento puede no ser tan bueno en corpus más pequeños.</li>
          <li>Entrenamiento más complejo y puede ser más lento que Word2Vec.</li>
          <li>Requiere una cantidad significativa de memoria y almacenamiento.</li>
        </ul>
      </td>
    </tr>
  </tbody>
</table>

### ++++++++ Termina la sección de agregar texto: +++++++++++


#**Pregunta - 6:**

Utiliza el modelo FastText de vectores embebidos pre-entrenados de dimensión 300 para generar un nuevo diccionario clave-valor, donde la “clave” será cada token o palabra de tu vocabulario y el “valor” será su vector embebido de dimensión 300.

Este diccionario deberá ser del mismo tamaño que el vocabulario previo que hayas construido previamente.

Es recomendable que una vez que generes el nuevo vocabulario de vectores embebidos, guardes dicho diccionario en un archivo.

Recuerda borrar la variable donde descargaste los 2 millones de vectores embebidos Fasttext.



In [ ]:
!pip install fasttext
import fasttext
import fasttext.util

fasttext.util.download_model('en',if_exists='ignore')

'cc.en.300.bin'

In [ ]:
# ******* Inlcuye a continuación todas las líneas de código y celdas que requieras: ***********
ft = fasttext.load_model('cc.en.300.bin')
vocab_embedings = {}

for token,idx in diccionario.items():
  vector = ft.get_word_vector(token)
  vocab_embedings[token] = vector

del ft

# Guardar el diccionario en un archivo
with open('vocabulario_fasttext.pkl', 'wb') as f:
    pickle.dump(vocab_embedings, f)

# Confirmar que se ha guardado
print("Diccionario guardado en 'vocabulario_fasttext.pkl'")

# *********** Aquí termina la sección de agregar código *************

Diccionario guardado en 'vocabulario_fasttext.pkl'


In [ ]:
# Cargar el diccionario desde el archivo
with open('vocabulario_fasttext.pkl', 'rb') as f:
    vocab_embedings = pickle.load(f)




In [ ]:
#Imprimir los primeros N elementos del diccionario
N = 1  # Número de elementos a imprimir
print(f"\nPrimeros {N} elementos del diccionario:")
for i, (token, c) in enumerate(vocab_embedings.items()):
    if i >= N:
        break
    print(f"Token: {token}, vector: {c}")
    print(f"vector sz {len(c)}")


Primeros 1 elementos del diccionario:
Token: be, vector: [-9.63205993e-02 -8.19036737e-02 -4.77219410e-02  1.46949440e-01
 -2.23413676e-01 -8.14826041e-02 -2.62784898e-01 -9.18371975e-03
  9.39049870e-02 -8.82623310e-04 -1.31746689e-02 -8.48824829e-02
 -5.35654761e-02 -6.28753332e-03 -1.85673028e-01  2.29752824e-01
  4.28092778e-02  4.56203744e-02 -2.81269066e-02  5.51726639e-01
 -7.04716146e-02  5.79598658e-02  3.89252491e-02 -3.09593510e-03
  1.17756665e-01 -1.82199806e-01  6.86402097e-02  3.97641845e-02
 -1.04433924e-01  3.45441222e-01 -5.88678569e-03 -8.50912184e-02
  6.21016659e-02  2.97347546e-01  2.68255733e-02  3.01602781e-02
 -1.00006558e-01 -1.55223802e-01 -1.22539923e-01  6.37540175e-03
 -1.39430225e-01  6.34008273e-02 -2.51493216e-01  5.28848097e-02
  1.95040017e-01 -1.30737111e-01 -5.72192073e-02  2.75106847e-01
  1.09033130e-01 -9.26046371e-02 -5.19613810e-02 -6.83219405e-03
 -8.30914751e-02 -1.54212385e-01 -1.24819856e-02  1.38309076e-01
 -3.80693078e-02 -3.24041903e-01

#**Pregunta - 7:**



Generamos los vectores embebidos a paertir de los conjuntos de entrenamiento, validación y preuba.

Los llamaremos trainEmb, valEmb y testEmb, respectivamente.

In [ ]:
# ******* Inlcuye a continuación todas las líneas de código y celdas que requieras: ***********
def tokens_a_embedding(lista_tokens, vocab_embeddings):
    vectores_palabras = [vocab_embeddings[token] for token in lista_tokens if token in vocab_embeddings]
    if vectores_palabras:
        return np.mean(vectores_palabras, axis=0)
    else:
        return np.zeros(300)  # Dimensión de los embeddings de FastText

# Convertir los enunciados en embeddings
def generar_embedings(sentences, vocab_embeddings):
    return np.array([tokens_a_embedding(lista_tokens, vocab_embeddings) for lista_tokens in sentences])


# Generar los embeddings para cada conjunto de datos
trainEmb =generar_embedings(x_train, vocab_embedings)
valEmb = generar_embedings(x_val, vocab_embedings)
testEmb = generar_embedings(x_test, vocab_embedings)

# *********** Aquí termina la sección de agregar código *************

In [ ]:
# Veamos las dimensiones de cada conjunto embebido:

print("Train-Emb:", trainEmb.shape)
print("Val-Emb:", valEmb.shape)
print("Test-Emb:", testEmb.shape)

Train-Emb: (2100, 300)
Val-Emb: (450, 300)
Test-Emb: (450, 300)


#**Pregunta - 8:**



Utiliza los modelos de regresión logística y bosque aleatorio (random forest) y encuentra sus desempeños.

Compara los resultados con los de la semana anterior.

In [ ]:
#Busqueda de hiperparametros para obtener un mejor modelo:
from scipy.stats import loguniform
from sklearn.model_selection import RandomizedSearchCV
#Hiperparámetros para Regresión Logística
hiperparametros_LR = {
    'C': loguniform(1e-5, 100),
    'penalty': [ None, 'l2',],
    'solver':  ['newton-cg', 'lbfgs', 'liblinear'],
    'max_iter': [1000]
}

#Inicializar modelos
modelo_LR = LogisticRegression()


#Inicializar búsqueda de cuadrícula para cada modelo con sus respectivos hiperparámetros
busqueda_LR = RandomizedSearchCV(modelo_LR, hiperparametros_LR, n_iter=500, random_state=248, n_jobs=-1 ,cv=5)

#Ajustar búsqueda de cuadrícula a los datos
busqueda_LR.fit(trainEmb, y_train)


#Obtener los mejores hiperparámetros para cada modelo
mejores_hiperparametros_LR = busqueda_LR.best_params_

print("Mejores hiperparámetros para Regresión Logística:", mejores_hiperparametros_LR)


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
425 fits failed out of a total of 2500.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
425 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py", line 1216, in fit
    self.coef_, self.intercept_, self.n_iter_ = _fit_liblinear(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py", line 1223, in _fit_liblinear
    solver_type = _get_liblinear_solver_type(m

Mejores hiperparámetros para Regresión Logística: {'C': 52.97573637713819, 'max_iter': 1000, 'penalty': 'l2', 'solver': 'liblinear'}


In [ ]:
# REGRESIÓN LOGÍSTICA:

# ******* Inlcuye a continuación todas las líneas de código y celdas que requieras: ***********


#Modelo de regresión logística
modeloLR = None
modeloLR = LogisticRegression( C= 52.97573637713819, max_iter=1000, solver= 'liblinear', penalty= 'l2')
modeloLR.fit(trainEmb, y_train)

print('LR: Train-accuracy: %.2f%%' % (100*modeloLR.score(trainEmb, y_train)))
print('LR: Val-accuracy: %2.f%%' % (100*modeloLR.score(valEmb, y_val)))

from sklearn.metrics import confusion_matrix

predLR = modeloLR.predict(testEmb)
print('\nMatriz de confusión con el mejor modelo de regrecion logistica:')
print(confusion_matrix(y_test, predLR, labels=[0,1]))


# *********** Aquí termina la sección de agregar código *************


LR: Train-accuracy: 87.81%
LR: Val-accuracy: 81%

Matriz de confusión con el mejor modelo de regrecion logistica:
[[180  36]
 [ 44 190]]


In [ ]:
#Busqueda de hiperparametros para obtener un mejor modelo RF:
from scipy.stats import loguniform
from sklearn.model_selection import RandomizedSearchCV
#Hiperparámetros para Random Forest
hiperparametros_RF = {
    'n_estimators': [50, 150, 100, 150, 200],
    'max_depth': [None, 2, 3, 5, 8, 13, 21],
    'min_samples_split': [2, 3, 5, 8, 13, 21],
    "criterion": ['gini', 'entropy', 'log_loss']
}


#Inicializar modelos
modelo_RF = RandomForestClassifier()


#Inicializar búsqueda de cuadrícula para cada modelo con sus respectivos hiperparámetros
busqueda_RF = RandomizedSearchCV(modelo_RF, hiperparametros_RF, n_iter=250, random_state=248, n_jobs=-1 ,cv=5)

#Ajustar búsqueda de cuadrícula a los datos
busqueda_RF.fit(trainEmb, y_train)


#Obtener los mejores hiperparámetros para cada modelo
mejores_hiperparametros_RF = busqueda_RF.best_params_

print("Mejores hiperparámetros para random forest:", mejores_hiperparametros_RF)

Mejores hiperparámetros para random forest: {'n_estimators': 200, 'min_samples_split': 8, 'max_depth': 13, 'criterion': 'entropy'}


In [ ]:
# BOSQUE ALEATORIO (Random Forest):

# ******* Inlcuye a continuación todas las líneas de código y celdas que requieras: ***********


# Crear y entrenar el modelo de bosque aleatorio
modeloRF = RandomForestClassifier(max_depth = 13, min_samples_split = 8, n_estimators = 200, criterion= 'entropy')
modeloRF.fit(trainEmb, y_train)

print('LR: Train-accuracy: %.2f%%' % (100*modeloRF.score(trainEmb, y_train)))
print('LR: Val-accuracy: %2.f%%' % (100*modeloRF.score(valEmb, y_val)))

from sklearn.metrics import confusion_matrix

predRF = modeloRF.predict(testEmb)
print('\nMatriz de confusión con el mejor modelo de random forest:')
print(confusion_matrix(y_test, predRF, labels=[0,1]))


# *********** Aquí termina la sección de agregar código *************

LR: Train-accuracy: 99.90%
LR: Val-accuracy: 79%

Matriz de confusión con el mejor modelo de random forest:
[[167  49]
 [ 50 184]]


#**Pregunta - 9:**



Reporte del mejor modelo.


In [ ]:
# ******* Inlcuye a continuación todas las líneas de código y celdas que requieras: ***********
from sklearn.metrics import classification_report

# Generar el reporte de clasificación LR
reporte_clasificacionLR = classification_report(y_test, predLR, target_names=['negativo', 'positivo'])

# Imprimir el informe de clasificación LR
print("Reporte de Clasificacion para regresion logistica:")
print(reporte_clasificacionLR)



# Generar el reporte de clasificación RF
reporte_clasificacionRF = classification_report(y_test, predRF, target_names=['negativo', 'positivo'])

# Imprimir el informe de clasificación FR
print("Reporte de Clasificacion para Random forest:")
print(reporte_clasificacionRF)


# *********** Aquí termina la sección de agregar código *************

Reporte de Clasificacion para regresion logistica:
              precision    recall  f1-score   support

    negativo       0.80      0.83      0.82       216
    positivo       0.84      0.81      0.83       234

    accuracy                           0.82       450
   macro avg       0.82      0.82      0.82       450
weighted avg       0.82      0.82      0.82       450

Reporte de Clasificacion para Random forest:
              precision    recall  f1-score   support

    negativo       0.77      0.77      0.77       216
    positivo       0.79      0.79      0.79       234

    accuracy                           0.78       450
   macro avg       0.78      0.78      0.78       450
weighted avg       0.78      0.78      0.78       450



#**Pregunta - 10:**



Incluye tus comentarios finales de la actividad.

### ++++++++ Inicia la sección de agregar texto: +++++++++++

Los modelos de regresión logística y random forest obtuvieron mejores resultados que en la ocasión anterior.

Sin embargo, estos modelos son muy simples para obtener resultados superiores a los obtenidos, lo que implica un límite en la exactitud que se puede alcanzar con ellos.

Aun así, estos modelos resultan ser muy útiles para este análisis, pues con la cantidad limitada de datos y el poder de procesamiento limitado, lograron obtener una exactitud respetable.

En varias ocasiones para este caso, la regresión logística resultó ser mejor que el bosque aleatorio, a diferencia de la ocasión anterior. Esto puede deberse a que trabajar ahora con vectores es más fácil para el modelo obtener una "dirección" que indica el sentido del comentario, a diferencia del modelo de random forest, que resultó ser mejor en la ocasión anterior al tomar decisiones sobre los valores de las matrices.

En cuanto al procesamiento de los datos, al usar lematización y convertir las contracciones comunes en el idioma inglés a las palabras completas, se pudo mejorar el desempeño de los modelos.


### ++++++++ Termina la sección de agregar texto: +++++++++++

##**Fin de la Actividad de vectores Embebidos - FastText**

Bibliografia:



*   FastText. (n.d.). https://fasttext.cc/
*   Google Code Archive - Long-term storage for Google Code Project Hosting. (n.d.). https://code.google.com/archive/p/word2vec/



